Jan 12 2021 (Tue)

# 카프카 내용
https://victorydntmd.tistory.com/344

In [1]:
# spark에서 카프카 데이터를 소비하는 곳

import sys
import os
import sqlite3
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext # 실시간 데이터 처리
from pyspark.streaming.kafka import KafkaUtils

In [2]:
# spark에는 kafka라이브러리가 없음 -> 생성
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.3.2 pyspark-shell'

# Spark Streaming
- 아주 작은 타임윈도우로 배치프로세스를 계속 실행하여 실시간처럼 보이는 기술


In [3]:
sc  = SparkContext( appName='streamTest' ) 
ssc = StreamingContext(sc,5) # 5초 주기로 pooling 

In [4]:
# sqlite db에 넣는 과정
def insertData( data ):
    sql = "insert into ktest values(?,?)"
    db = sqlite3.connect(r'C:\Users\master4\Documents\eunseo\my.db')
    db.execute( sql, data  )  
    db.commit() # insert, delete는 무조건 commit해야한다.
    db.close()

In [5]:
# 콜백함수 등록
# topic이 있으면 이 함수가 호출된다. 
def recvData(data):
    insertData( data )    # insertData 함수 호출하고 data를 삽입
    print('data=', data ) # (키, 값) 튜플형태로 
    return data  

In [6]:
topic = 'sample'
kvs = KafkaUtils.createStream( ssc, 
                              'localhost', # kafka cluster
                              'raw-event-streaming-consumer',
                              {topic:1} )
#========================
#메시지 수신시 recvData call
lines = kvs.map( recvData ) # 튜플형태로 받아온다
lines.pprint() #상태메시지
#==============================
ssc.start() # 카프카 클러스터로부터 sample topic이 들어왔는지 5초에 한번씩 풀링
ssc.awaitTerminationOrTimeout(2) # 2초 뒤에 중지하겠다

-------------------------------------------
Time: 2021-01-12 13:53:05
-------------------------------------------



False

-------------------------------------------
Time: 2021-01-12 13:53:10
-------------------------------------------

-------------------------------------------
Time: 2021-01-12 13:53:15
-------------------------------------------

-------------------------------------------
Time: 2021-01-12 13:53:20
-------------------------------------------

-------------------------------------------
Time: 2021-01-12 13:53:25
-------------------------------------------

-------------------------------------------
Time: 2021-01-12 13:53:30
-------------------------------------------

-------------------------------------------
Time: 2021-01-12 13:53:35
-------------------------------------------

-------------------------------------------
Time: 2021-01-12 13:53:40
-------------------------------------------

-------------------------------------------
Time: 2021-01-12 13:53:45
-------------------------------------------

-------------------------------------------
Time: 2021-01-12 13:53:50
----------

In [7]:
ssc.stop() # 폴링 중지 